In [ ]:
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Flatten, Concatenate
from gee_scripts.parameters import explain_vars, response_var
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
df = pd.read_csv("data/9_clean_training_data/clean_training_data.csv")
response_var = response_var[0]

In [ ]:
# LSTM Sequence Input
sequence_input_1 = Input(shape=(4, 1), name='sequence_input')  # 4 time steps, 1 feature (sm value)
lstm_out_1 = LSTM(50)(sequence_input_1)  # 50 LSTM units, can be tuned

sequence_input_2 = Input(shape=(4, 1), name='sequence_input')  # 4 time steps, 1 feature (sm value)
lstm_out_2 = LSTM(50)(sequence_input_2)  # 50 LSTM units, can be tuned

# Dense Input for non-sequential data
dense_input = Input(shape=(26,), name='dense_input')  # 26 other explanatory variables
dense_out = Dense(50, activation='relu')(dense_input)  # 50 units, can be tuned

# Combine LSTM and Dense outputs
merged = Concatenate()([lstm_out, dense_out])

# Add further dense layers if needed
dense_merged_1 = Dense(100, activation='relu')(merged)
dense_merged_2 = Dense(50, activation='relu')(dense_merged_1)

# Regression output
output = Dense(1, activation='linear')(dense_merged_2)

# Compile the model
model = Model(inputs=[sequence_input, dense_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Using MSE loss for regression

# Print model summary to see the architecture
# model.summary()


In [ ]:

# Assuming data_X contains your input features and data_y contains the target values
data_X = df.drop(response_var, axis=1)
data_y = df[response_var]

X_train, X_temp, y_train, y_temp = train_test_split(data_X, data_y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
sm_vars = ["sm_3", "sm_7", "sm_30"]
pr_vars = ["prec_3", "prec_7", "prec_30"]
dense_vars = [exp for exp in explain_vars if exp not in sm_vars + pr_vars]

X_train_lstm_1 = X_train[sm_vars].values.reshape(-1, 3, 1)
X_train_lstm_2 = X_train[pr_vars].values.reshape(-1, 3, 1)
X_train_dense = X_train[dense_vars].values.reshape(-1, len(dense_vars))

In [ ]:
model.fit([X_train_lstm, X_train_dense], y_train, validation_data=([X_val_lstm, X_val_dense], y_val), epochs=50, batch_size=32)

In [ ]:
y_pred = model.predict([X_test_lstm, X_test_dense])

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

In [ ]:

# Scatter plot of actual vs. predicted
plt.figure(figsize=(10,6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')  # 45-degree line
plt.xlabel("Actual Groundwater Levels")
plt.ylabel("Predicted Groundwater Levels")
plt.title("Actual vs. Predicted")
plt.show()

# Residual plot
residuals = y_test - y_pred
plt.figure(figsize=(10,6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.plot([min(y_pred), max(y_pred)], [0, 0], color='red')
plt.xlabel("Predicted Groundwater Levels")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()